In [2]:

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
import numpy as np
import psutil
import time
import pickle


In [7]:

class pilotnet():
    def __init__(self) -> None:
        self.weight_set=False
        self.partition_done=False
        self.input_loaded=False
        self.layer_list=[]
        
        self.model = models.Sequential()
        # # Normalization layer
        self.model.add(layers.LayerNormalization(center=True , scale=True,input_shape=(66,200, 3)))

        # Convolutional layers
        self.model.add(layers.Conv2D(24, (5, 5), strides=(2, 2), activation='relu'))
        self.model.add(layers.Conv2D(36, (5, 5), strides=(2, 2), activation='relu'))
        self.model.add(layers.Conv2D(48, (5, 5), strides=(2, 2), activation='relu'))
        self.model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        self.model.add(layers.Conv2D(64, (3, 3), activation='relu'))

        # Flatten layer
        self.model.add(layers.Flatten())

        # Fully connected layers
        self.model.add(layers.Dense(100, activation='relu'))
        self.model.add(layers.Dense(50, activation='relu'))
        self.model.add(layers.Dense(10, activation='relu'))
        
        # Output layer
        self.model.add(layers.Dense(1))  # Output: steering angle
        self.model.compile(optimizer='adam', loss='mse', metrics=['mae'])
       
        #install "pip install pydot-ng"  and "apt install graphviz" for ploting below model
        tf.keras.utils.plot_model(self.model, to_file='pilotnet_model.png', show_shapes=True)
        
        # return self.model
        
    def loadWeights(self):
        self.model.load_weights('./pilotnet_model.h5')
        # model = load_model('model.h5')
        self.weight_set=True
        print("\_____Weights are loaded to the ")
        
    def model_info(self):
        self.no_of_layers=len(self.model.layers)
        print("\_____Number of Layers in Model: ",self.no_of_layers)
        print("\_____Weights are loaded: ",self.weight_set)
        print("\_____Input loaded to Model: ",self.input_loaded)
        print('\_____Partitioning Done: ',self.partition_done)
        
    def load_input(self):
        self.input_data = np.random.rand(1, 66, 200, 3).astype(np.float32)
        self.input_loaded=True
        return self.input_data
        
    def make_partition(self):
        self.NO_OF_LAYERS= len(self.model.layers)
        
        for i in range(self.NO_OF_LAYERS):
            self.temp_layer=self.model.layers[i]
            self.layer_list.append(self.temp_layer)
            
        self.partition_done = True
        print('\_______Partitioning Done')

    
    def save_pickeled_layers(self):
        if not self.weight_set:
            self.loadWeights()

        
        if not self.partition_done:
            self.make_partition()
        save_dir='pilotnet_pickle'
        for i in range(len(self.layer_list)):
            fname=f'./{save_dir}/pilotnet_layer_{i}.pkl'
            layer_weights_and_config = {
                'weights': self.layer_list[i].get_weights(),
                'config': tf.keras.layers.serialize(self.layer_list[i])}
            with open(fname, 'wb') as f:
                pickle.dump(layer_weights_and_config, f)
    
    def execute_full_network(self):
        if not self.input_loaded:
            self.load_input()
        print("I am in full Executioon")
        st1=time.perf_counter()
        self.output=self.model.predict(self.input_data)
        ed1=time.perf_counter()
        
        elt1=ed1-st1
        elt1=float(elt1)
        return elt1,self.output
        
    def execute_full_network_sample(self,imp):
        st1=time.perf_counter()        
        self.output=self.model.predict(imp)
        ed1=time.perf_counter()
        
        elt1=ed1-st1
        elt1=float(elt1)        
        return elt1
    
    def execute_full_partition(self):
        if not self.partition_done:
            self.make_partition()
        if not self.input_loaded:
            self.load_input()
            
        self.temp_res=self.input_data
        st2=time.perf_counter()
        for i in range(self.NO_OF_LAYERS):
            self.temp_res = self.layer_list[i](self.temp_res)
        ed2=time.perf_counter()
        self.temp_res=np.array(self.temp_res)
        elt2=ed2-st2
        elt2=float(elt2)
        
        return elt2,self.temp_res
    
    def execute_layer_by_layer_sample(self,inp):
        temp_out=inp
        for lay in self.layer_list:
            
            temp_out=lay(temp_out)
               
        temp_out=np.array(temp_out)
                    
        return temp_out
    
    def execute_on_core(self,layer_id,input_data):
                
        self.temp_out=self.layer_list[layer_id](input_data)
        
        return self.temp_out
    
    

In [8]:
in_d=np.zeros(shape=(1,66, 200,3)).astype(np.float32)


In [9]:
np.random.seed(42)

# try:
#     psutil.Process().cpu_affinity([121])
# except AttributeError:
#     pass 

# obj =vgg16_in()
# obj.loadWeights()
# obj.model_info()
obj=pilotnet()
obj.loadWeights()
obj.model_info()
obj.make_partition()
res1=obj.execute_full_network()
# res2 = obj.execute_full_partition()
print(f'Time required : {res1[0]} result : ')
# print(f'Time required : {res2[0]} result : ')


/home/ankit/anaconda3/envs/nnsc/lib/python3.11/site-packages/keras/src/layers/normalization/layer_normalization.py:122: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


\_____Weights are loaded to the 
\_____Number of Layers in Model:  11
\_____Weights are loaded:  True
\_____Input loaded to Model:  False
\_____Partitioning Done:  False
\_______Partitioning Done
I am in full Executioon
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Time required : 0.08076475799998661 result : 


In [10]:
obj.model_info()
obj.save_pickeled_layers()

\_____Number of Layers in Model:  11
\_____Weights are loaded:  True
\_____Input loaded to Model:  True
\_____Partitioning Done:  True
